In [9]:
import tensorflow as tf
import numpy as np
import mnist
import matplotlib.pyplot as plt
import IPython.display as display

In [3]:
model = tf.keras.Sequential(
  [
      tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
      
      tf.keras.layers.Conv2D(kernel_size=3, filters=12, use_bias=False, padding='same'),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Conv2D(kernel_size=6, filters=24, use_bias=False, padding='same', strides=2),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Conv2D(kernel_size=6, filters=32, use_bias=False, padding='same', strides=2),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Flatten(),
      
      tf.keras.layers.Dense(200, use_bias=False),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 12)        108       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 12)       36        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 24)        10368     
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 24)       72        
 hNormalization)                                        

In [4]:
train_digit, train_label = mnist.extract_train()
test_digit, test_label = mnist.extract_test()

train_digit = train_digit.reshape(-1,28*28)
test_digit = test_digit.reshape(-1,28*28)

train_label = tf.one_hot(train_label, 10)
test_label = tf.one_hot(test_label, 10)

print([tmp.shape for tmp in [train_digit, train_label, test_digit, test_label]])

In [25]:
BATCH_SIZE = 128
EPOCHS = 10
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset


print("Steps per epoch: ", steps_per_epoch)
history = model.fit(x = train_digit, y = train_label , steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=(test_digit, test_label), validation_steps=1)

Steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 60s 124ms/step - loss: 0.1394 - accuracy: 0.9594 - val_loss: 0.0578 - val_accuracy: 0.9814
Epoch 2/10
468/468 [==============================] - 55s 118ms/step - loss: 0.0436 - accuracy: 0.9870 - val_loss: 0.0673 - val_accuracy: 0.9798
Epoch 3/10
468/468 [==============================] - 54s 116ms/step - loss: 0.0307 - accuracy: 0.9907 - val_loss: 0.0300 - val_accuracy: 0.9904
Epoch 4/10
468/468 [==============================] - 54s 116ms/step - loss: 0.0241 - accuracy: 0.9927 - val_loss: 0.0326 - val_accuracy: 0.9891
Epoch 5/10
468/468 [==============================] - 63s 135ms/step - loss: 0.0178 - accuracy: 0.9946 - val_loss: 0.0279 - val_accuracy: 0.9898
Epoch 6/10
468/468 [==============================] - 60s 128ms/step - loss: 0.0165 - accuracy: 0.9949 - val_loss: 0.0273 - val_accuracy: 0.9914
Epoch 7/10
468/468 [==============================] - 77s 165ms/step - loss: 0.0133 - accuracy: 0.9959 - val

468/468 [==============================] - 68s 145ms/step - loss: 0.0093 - accuracy: 0.9968 - val_loss: 0.0239 - val_accuracy: 0.9923


In [26]:
model.save('saved_model/local_model')
model = tf.keras.models.load_model('saved_model/local_model')

INFO:tensorflow:Assets written to: saved_model/local_model\assets


INFO:tensorflow:Assets written to: saved_model/local_model\assets


In [27]:
digits, labels = mnist.extract_test()
digits = digits.reshape(-1,28*28)

pred = model.predict(digits)
pred = [np.argmax(p) for p in pred]
pred = np.array(pred)

res = pred == labels
acc = res[res].sum() / res.shape[0] * 100
acc

313/313 [==============================] - 5s 14ms/step


99.22999999999999